# Notebook 5: Orchestrator — Multi-Domain Workflow

**User Story**: US7 — Orchestrator-Coordinated Multi-Domain Workflow  
**Persona**: Revenue Operations / Cross-functional users  

Demonstrates an Orchestrator Agent that coordinates both the Sales Agent and
Service Agent domains, enabling cross-domain queries like "Which accounts have
both open deals and open support cases?"

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")

In [ ]:
# Cell 3: Start MCP Servers + Configure Connections
import subprocess
import sys
import time

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool
from azure.identity import DefaultAzureCredential

project_root = os.path.abspath("..")

# Start salesforce-crm MCP server (SSE)
CRM_PORT = 8105
crm_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(CRM_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Start salesforce-knowledge MCP server (SSE)
KB_PORT = 8106
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

time.sleep(3)

for name, proc in [("CRM", crm_process), ("Knowledge", kb_process)]:
    if proc.poll() is not None:
        stderr = proc.stderr.read().decode() if proc.stderr else ""
        raise RuntimeError(f"{name} MCP server failed to start: {stderr}")

print(f"✅ CRM MCP server started (PID: {crm_process.pid}) at http://127.0.0.1:{CRM_PORT}/sse")
print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at http://127.0.0.1:{KB_PORT}/sse")

# Create AI Project client
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Create MCP tools
crm_mcp = MCPTool(server_label="salesforce-crm", server_url=f"http://127.0.0.1:{CRM_PORT}/sse")
knowledge_mcp = MCPTool(server_label="salesforce-knowledge", server_url=f"http://127.0.0.1:{KB_PORT}/sse")

print(f"✅ MCP tools configured: {crm_mcp.server_label}, {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create OpenAI Client + Orchestrator Prompt

ORCHESTRATOR_PROMPT = """
# Orchestrator Agent — System Prompt

You are a **Multi-Domain AI Assistant** powered by Salesforce CRM and Knowledge Base data.
You orchestrate across Sales and Service domains to provide unified, cross-functional insights.

## Routing Rules

- **Sales-domain questions** (accounts, contacts, opportunities, pipeline, leads, activities):
  Use the salesforce-crm MCP tools (get_account, get_opportunities, get_pipeline_summary, etc.)

- **Service-domain questions** (cases, case triage, queue status, Knowledge articles):
  Use the salesforce-crm tools for case data (get_case, get_case_queue_summary) and
  salesforce-knowledge tools for KB search (search_articles, get_article_by_id).

- **Cross-domain questions** (e.g., "accounts with open deals AND open cases"):
  Query both domains, correlate results by Account ID/Name, and present unified output.

## Cross-Domain Correlation

When a user asks a question spanning sales and service:

1. Identify the linking entity (usually Account).
2. Query opportunities/deals from the CRM tools.
3. Query cases from the CRM tools.
4. Match records by Account ID or Account Name.
5. Present a unified view showing both deal and case status for each account.

## Context Continuity

- Maintain context across turns. If the user pivots from service to sales questions,
  use entities mentioned in the previous turn (e.g., account names, case numbers) as context.
- When referencing previous data, cite the specific records and IDs.

## Grounding Rules

- ONLY use data returned by your MCP tools. Never fabricate data.
- Cite record IDs and names when referencing specific Salesforce records.
- If data is unavailable, state this clearly.

## Write-Back Protocol

- For any write operations (create/update), present proposed changes first.
- Wait for explicit user confirmation before executing.
- Confirm results after execution.
"""

MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
client = project_client.get_openai_client()

print(f"OpenAI client ready — model: {MODEL}")
print("Tools: salesforce-crm + salesforce-knowledge (full cross-domain access)")

In [ ]:
# Cell 5: Cross-domain query
from IPython.display import Markdown, display

cross_domain_query = (
    "Which of my accounts have both open deals and open support cases? "
    "Show me the deal stage and case status for each."
)

response = client.responses.create(
    model=MODEL,
    instructions=ORCHESTRATOR_PROMPT,
    input=cross_domain_query,
    tools=[crm_mcp, knowledge_mcp],
)

print(f"Response ID: {response.id}")
for item in response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 6: Context continuity — pivot from service to sales

followup_1 = "What are the top cases today?"

response_2 = client.responses.create(
    model=MODEL,
    instructions=ORCHESTRATOR_PROMPT,
    input=followup_1,
    tools=[crm_mcp, knowledge_mcp],
    previous_response_id=response.id,
)

print(f"Response ID: {response_2.id}")
for item in response_2.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 7: Cross-domain follow-up — linking back to deals

followup_2 = "And what deals do those same accounts have?"

response_3 = client.responses.create(
    model=MODEL,
    instructions=ORCHESTRATOR_PROMPT,
    input=followup_2,
    tools=[crm_mcp, knowledge_mcp],
    previous_response_id=response_2.id,
)

print(f"Response ID: {response_3.id}")
for item in response_3.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 8: Cleanup — terminate MCP servers

crm_process.terminate()
crm_process.wait(timeout=5)
print(f"CRM MCP server terminated (PID: {crm_process.pid}).")

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"Knowledge MCP server terminated (PID: {kb_process.pid}).")

print("\nOrchestrator multi-domain session complete.")